# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [3]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 400)
import pickle

# magic word for producing visualizations in notebook
%matplotlib inline


## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
#azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
#customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

In [ ]:
# Dump the azdias dataframe to a pickle object 
#pickle.dump(azdias, open("azdias.pickle", "wb"))

In [ ]:
# Reload azdias object 
azdias = pickle.load(open("azdias.pickle", "rb"))

In [ ]:
azdias.head()

In [ ]:
#from helpers import save_csv
#helpers.save_csv(azdias, 'data_project', 'Udacity_AZDIAS_052018.csv')
#helpers.save_csv(customers, 'data_project', 'Udacity_CUSTOMERS_052018.csv')


## Part 0.1 Data exploration 

In [ ]:
#azdias = pd.read_csv('./data_project/Udacity_AZDIAS_052018.csv')
#azdias.drop(['Unnamed: 0'], axis = 1, inplace = True)
#azdias.head()

In [ ]:
#customers = pd.read_csv('./data_project/Udacity_CUSTOMERS_052018.csv')
#customers.head()

In [ ]:
print('Describe Customers :\n')
display(customers.describe())

print('Describe Population :\n')
display(azdias.describe())

In [ ]:
# The shape of dataframes
print('Shape of azdias dataframe: {}\n'.format(azdias.shape))
print('Shape of customers dataframe: {}\n'.format(customers.shape))

### check if azdias contains all columns in customers

In [ ]:
# check if azdias contains all columns in customers

customers_columns_list = list(customers.columns)
azdias_columns_list = list(azdias.columns)

result =  all(elem in customers_columns_list  for elem in azdias_columns_list)
 
if result:
    print("Yes, customers dataframe contains all columns in azdias dataframe")    
else :
    print("No, customers dataframe does not contain all columns in azdias dataframe")

### Data_info and Mising values

Now we will explore a little deeper the data. Let's load the data_info.csv file so that we will have more information about the columns. This would help us in our data cleaning processing.

In [ ]:
data_info = pd.read_csv('data_info.csv')
data_info.drop(['Attribute','ISBLANK'], axis = 1, inplace = True)
data_info.rename(columns={"Attribute.1": "Attribute", "Missing_value": "Missing_Value"}, inplace = True)

data_info.head(10)

In [ ]:
len(data_info.Attribute.unique())

In [ ]:
# Unique code for missing values
data_info.Missing_Value.unique()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

### Part 1.1  Check for Missing Values  - azdias df 

We can notice that we have 273 columns with missing data. We can also see below that even before the missing data mapping process, we have few columns with high percentage of missing values, such as ALTER_KIND4 (99.86%), ALTER_KIND3 (99.31%), ALTER_KIND2 (96.69%),ALTER_KIND1 (90.90%), EXTSEL992(73.40%), KK_KUNDENTYP (65.60%).



In [ ]:
# columns with missing values
missing_data = azdias.isnull().sum()[azdias.isnull().sum() !=0]

# dictionary with missing values and their respective percentage 

total_rows = azdias.shape[0]
missing_dict = {'Missing_Count': missing_data.values, 
                'Percentage': np.round(missing_data.values*100/(total_rows),2)}


#Create DataFrame
azdias_missing_initial = pd.DataFrame(missing_dict, index = missing_data.index)
azdias_missing_initial.sort_values(by = 'Missing_Count', ascending = False, inplace = True)
print('Number of columns with missing values: ', azdias_missing_initial.shape[0])
display('     azdias_missing_initial DataFrame: ')
azdias_missing_initial


In [ ]:
# Double-check
print('Percentage of missing values (ALTER_KIND4): ', round(100*azdias.isnull().sum()['ALTER_KIND4']/ total_rows,2))
print('Percentage of missing values (ALTERSKATEGORIE_FEIN): ',round(100*azdias.isnull().sum()['ALTERSKATEGORIE_FEIN']/ total_rows,2))

### Create dataframe with attributes and the missing codes from data_info.csv file : 

In [ ]:
missing_code_df = data_info.drop(['Description','Value','Meaning'], axis = 1)
#missing_code_df.set_index('Attribute', inplace = True)

#missing_code_df['Missing_value'] = missing_code_df['Missing_value'].str.replace('[','').replace(']','').split(',').values


display('Missing codes and dtype: ',missing_code_df.Missing_Value.unique())
print()
missing_code_df.head()

In [ ]:
len(missing_code_df.Attribute.unique())

In [ ]:

missing_code_df.drop_duplicates(keep='first',inplace=True) 
display(len(missing_code_df.Attribute.unique()))

missing_code_df.head()

In [ ]:
len(missing_code_df.Attribute)

In [ ]:
missing_code_df.isnull().sum()

In [ ]:
# Some variables are not categorical, and have no missing value code
# let's give them a -1 code so that we can use the built in function eval() to build our dictionary with int codes
missing_code_df.fillna('[-1]', inplace = True)

In [ ]:
# Create a dictionary with attribute as key and its respective missing code as value
from helpers import create_missing_code_dict
missing_dict = create_missing_code_dict(missing_code_df)
#missing_dict   

I noticed that there are attributes in the azdias dataframe that are missing in the data_info dataframe. To be more specific, we have  93 missing Attributes and because of that we don't have information regarding their missing code.So, I will only consider nan as missing value for those attributes( I could try to consider 0 as missing value but I have no information regarding those features) 

In [ ]:
missing_column_count =0
present_column_count = 0
missing_column_list = []

for col in azdias.columns[1:]:
    if col not in missing_dict:
        missing_column_list.append(col)
        missing_column_count += 1
        pass
    else:
        present_column_count +=1
print('Number of columns of azdias df that are missing in data_info df: ', missing_column_count)
print('Number of columns of azdias df contained in data_info df: ', present_column_count)

In [ ]:
# Calculate number missing columns that have 0 (potencial missing value)
# NOTE : I am not considering 0 as missing value. This is just a test
zero_one_missing_code = []
for col in missing_column_list:
   # for val in azdias[col].value_counts().index:
        if 0 in azdias[col].value_counts().index:
           zero_one_missing_code.append(col)
print('Number of columns that contains 0: ',len(zero_one_missing_code))       

In [ ]:
from helpers import valid_values_dict
valid_values_dict = valid_values_dict(azdias, missing_dict)

 We can see below that the missing codes in the azdias dataframe has been converted to nan values.

In [ ]:
# Dataframe with missing codes converted to nan
azdias_after_encoding = azdias.copy()


# skip LNR column
for col in azdias.columns[1:]:  

    azdias_after_encoding[col] = azdias_after_encoding[col].map(valid_values_dict[col])

azdias_after_encoding.head()

In [ ]:
azdias.head()

In [ ]:
# total of missing values per columns after converting codes
total_nan_after_encoding = azdias_after_encoding.isnull().sum()
# total of missing values per columns before converting codes
total_nan_before_encoding = azdias.isnull().sum()

#difference

diff = total_nan_after_encoding - total_nan_before_encoding

#

In [ ]:
print('Missing values before encoding:', total_nan_before_encoding.sum())
print('Missing values after encoding:', total_nan_after_encoding.sum())
print('Total increase after encoding:', diff.sum())

From now on we can work on the new dataframe.

In [ ]:
# Dump the azdias_after_encoding dataframe to a pickle object since it takes up so much room in memory.
#pickle.dump(azdias_after_encoding, open("azdias_after_encoding.pickle", "wb"))

### Analysing Missing values

In [ ]:
#azdias_after_encoding  = pickle.load(open("azdias_after_encoding.pickle", "rb"))

In [ ]:
missing_dict = {'Before_Encoding': total_nan_before_encoding[1:],
               'After_Enconding': total_nan_after_encoding[1:],
               'Difference':diff[1:], 
               'Percent_nan': 100* total_nan_after_encoding/ azdias.shape[0]}

missing_df = pd.DataFrame(missing_dict)
missing_df.head(10)

In [ ]:
from helpers import missing_values_barplt
missing_values_barplt(missing_df,'Before_Encoding')

In [ ]:
missing_values_barplt(missing_df,'After_Enconding')


In [ ]:
missing_values_barplt(missing_df,'Difference')

In [ ]:
from helpers import missing_values_barplt
missing_values_barplt(missing_df,'Percent_nan')

The bar plots above show us that there are attributes with a lot of missing values after converting the missing value codes to nan variables. It's important to notice that some columns got a significant increase in missing values. Besides, the column AGER_TYP that has no missing values before the encoding, got tons of nan variables.

I will now check the distribution of the some attributes in order to find potencial outliers.


In [ ]:
missing_more_than_40_pct = missing_df.loc[missing_df.Percent_nan >= 40, :].sort_values(by = ['Percent_nan'], ascending = False)

In [ ]:
missing_more_than_40_pct

In [ ]:
# Missing values histogram
from helpers import hist_missing_values
hist_missing_values(missing_df )

In [ ]:
# missing values < 30 %
hist_missing_values(missing_df,threshold = 30,greater_or_less ='less' )

In [ ]:
# missing values > 30%
hist_missing_values(missing_df,threshold = 40,greater_or_less ='greater' )

In [ ]:
threshold = [10,20,30,40,50,50,60]
print('Missing Values Per Features:')
print()
for i in threshold:
    outliers = missing_df[missing_df['Percent_nan'] > i]
    print('Features with more than {}% of missing values: {}'.format(i,outliers.shape[0]))

In [ ]:
azdias_after_encoding.head()

In [ ]:
azdias.head()

## Removing Features

The distributions above show us that most of the features of the dataset have zero or small amount of missing values. In addition to that, we can notice that 35 featues have more than 40% of missing values. It seems reasonable to delete those columns since those features with high amount of missing values do note give us much information.


In [ ]:
missing_more_than_40_pct.head()


In [ ]:
print('features with more than 40% missing values:',missing_more_than_40_pct.shape[0])

In [ ]:
azdias.shape

In [ ]:
azdias_after_encoding.shape

In [ ]:
missing_more_than_40_pct.head()

In [ ]:
print('Dataframe before the columns removal:',azdias_after_encoding.shape)

In [ ]:
columns_to_drop = [x for x in missing_more_than_40_pct.index]
azdias_dropped_features = azdias_after_encoding.drop(columns_to_drop, axis =1 )

In [ ]:
columns_to_drop

In [ ]:
print('Dataframe after the columns removal:',azdias_dropped_features.shape) 

### Distribution with number of missing values per row after the outlier columns removal


In [ ]:
fig = plt.figure(figsize=(20,7))
azdias_dropped_features.isnull().sum(axis=1).hist(bins=20)
plt.title('Missing Values per Row');
plt.xlabel('Number of Features')
plt.ylabel('Number of Rows');

In [ ]:
# percentage of missing value for rows with about 240 nan variables
round(250/azdias_dropped_features.shape[1],2)

Let's now consider the number of missing values per row. Looking at the distribution above, we can see that there are rows with more than 250 missing values. This represents about 76% of nan varibles in the rows. So, in order to analyse those rows I will split the dataframe into two. The first will have less missing values than a specified threshold and the second will have the number of missing values greater than this threshold. Than, I will analyse the distribution of those groups and check if they are too different from one another. If so,I will be comfortable dropping those rows from my dataframe.


In [ ]:
# Split dataframe 
df_new = azdias_dropped_features.copy()

df_low_missing_values = df_new.dropna(thresh= 250) # Keep only the rows with at least 250 non-NA values

df_high_missing = df_new[~df_new.index.isin(df_low_missing_values.index)]

df_low_missing_values.shape, df_high_missing.shape 

In [ ]:
def compare_df_plot(df_low_missing, df_high_missing, column_names):
    
    for column_name in column_names:
    
        fig = plt.figure(figsize = (15,5))
        ax1 = fig.add_subplot(121)
        ax1.title.set_text('Low Missing Values')
        sns.countplot(df_low_missing.loc[:, column_name])

        ax2 = fig.add_subplot(122)
        ax2.title.set_text('High Missing Values')
        sns.countplot(df_high_missing.loc[:,column_name])      
        
        #fig.subtitle(column_name)
        plt.plot()

In [ ]:
columns = ['ANREDE_KZ','FINANZTYP','ZABEOTYP', 'SEMIO_TRADV', 'FINANZ_VORSORGER', 'FINANZTYP', 'ALTERSKATEGORIE_GROB', 'ONLINE_AFFINITAET']

In [ ]:
compare_df_plot(df_low_missing_values , df_high_missing ,columns)

In [ ]:
import random
#sampling with replacement

sampling = random.choices(azdias_dropped_features.columns, k=15)
print("Randomly selected multiple columns: \n ", sampling)

In [ ]:
compare_df_plot(df_low_missing_values , df_high_missing ,sampling)

We can notice that some features  have distribution completely different from the dataframe with smaller amount of missing value.This suggests that these two groups are different from one another.Based on that, from now on, I will keep working with the low_missing_values data frame.

## Feature Engineering

The unsupervised learning algorithm that will be used to build the customer segmentation, requires numerical values. Because of that, all the data must be numeric encoded so that the model can proceed the way it is supposed to proceed.



In [ ]:
# drop LNR columns - it looks to be just an identifier 
#df_low_missing_values.drop(['LNR'] ,axis =1 , inplace = True)

The above dictionaries contains the binary features and the multiple features. This helps to identify the columns that will need to be better investigated so that we can encode them correctly.

In [ ]:
def unique_values_dict(df):
    
    binary_variable = {}
    multiple_categorical_feature = {}

    for column in df.columns:

        unique_values = df[column].nunique()

        if unique_values <= 2:

            binary_variable[column] = unique_values

        else:
            multiple_categorical_feature[column] = unique_values

    return binary_variable, multiple_categorical_feature
    

    


In [ ]:
binary_variable , multiple_categorical_feature = unique_values_dict(df_low_missing_values)


In [ ]:
print('============ BINARY FEATURES: =============\n')
print('\n'.join('column: {} Unique Values: {}'.format(key,value) for key,value in binary_variable.items()))




In [ ]:
df_low_missing_values.OST_WEST_KZ.unique()

The only benary feature that needs to be encoded is OST_WEST_KZ
* array(['W', 'O'], dtype=object)

In [ ]:
print('========= MULTIPLE VALUES FEATURES: ==========\n')
print('\n'.join('column: {} Unique Values: {}'.format(key,value) for key,
                value in multiple_categorical_feature.items()))

In [ ]:
np.sort(df_low_missing_values.AKT_DAT_KL.unique())

In [ ]:
#Object features 
obj_features = df_low_missing_values.select_dtypes(include = ['object']).columns
obj_features

In [ ]:
# Unique values fperfeature
for k,v in multiple_categorical_feature.items():
    
    print(k,'\n','Unique value :', end=' ')
    
    if k in list(obj_features): # skip object type features 
        pass
    else:
        print(np.sort(df_low_missing_values[k].unique()))
        
    
    

Analysing each feature and its unique values, we can see that most of the features are already encoded.Even though there are ordinal and nominal variables in the dataset, it seems that categorizing them as been of the same type (categorical) will not cause huge inpact on the model. Becaus of that, I will assume all variables as being categorical. 

### Object Features

In [ ]:
#Object features 
obj_features

**After analysing all of those features, I decided to do as follows:**
* **OST_WEST_KZ** binary feature that will bee re-encoded to 0 and 1
* LNR - It seems to be a index so we can get rid of it
* **CAMEO_DEU_2015** - This feature has many rows with 'XX' category (347 rows). I will consider those inputs as missing values and replace with a numerical value
* **CAMEO_DEUG_2015** - Similarly the CAMEO_INTL_2015, this feature has many 'X' inputs. I will do the same as the above feature
* **CAMEO_INTL_2015** - Same as above feature
* **D19_LETZTER_KAUF_BRANCHE** - This featur has 36 categories (strings) and needs to be encoded
* **EINGEFUEGT_AM** - It is a date format feature and needs to be re-enconded (year e/or month)

In [4]:
from helpers import create_missing_code_dict
from helpers import valid_values_dict
def clean_df(df, missing_code_df, customer_data = False):
    
    print('====== Delete CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP features if customer_df ====')
    
    if customer_data:
        df.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis = 1, inplace = True)
        
    
    print('====== Drop index LNR ====== ')
    
    df.drop(['LNR'], axis = 1, inplace = True)
    
    print('====== Converte Missing Code ======')
    

    missing_dict = create_missing_code_dict(missing_code_df)
    
  
    valid_values_dict_ = valid_values_dict(df, missing_dict)
    
    # Dataframe with missing codes converted to nan
    #df_copy = df.copy()

   
    for col in df.columns: 

        df[col] = df[col].map(valid_values_dict_[col])
        
    print('====== Drop Features with more than 40% of missing values=====')
    
    columns_to_drop = ['ALTER_KIND4',
                         'TITEL_KZ',
                         'ALTER_KIND3',
                         'D19_TELKO_ONLINE_DATUM',
                         'D19_BANKEN_OFFLINE_DATUM',
                         'ALTER_KIND2',
                         'D19_TELKO_ANZ_12',
                         'D19_BANKEN_ONLINE_QUOTE_12',
                         'D19_BANKEN_ANZ_12',
                         'D19_TELKO_ANZ_24',
                         'D19_VERSI_ANZ_12',
                         'D19_TELKO_OFFLINE_DATUM',
                         'ALTER_KIND1',
                         'D19_BANKEN_ANZ_24',
                         'D19_VERSI_ANZ_24',
                         'D19_BANKEN_ONLINE_DATUM',
                         'GREEN_AVANTGARDE',
                         'D19_BANKEN_DATUM',
                         'AGER_TYP',
                         'D19_VERSAND_ONLINE_QUOTE_12',
                         'D19_TELKO_DATUM',
                         'EXTSEL992',
                         'D19_GESAMT_ONLINE_QUOTE_12',
                         'D19_VERSAND_ANZ_12',
                         'D19_VERSAND_OFFLINE_DATUM',
                         'D19_GESAMT_ANZ_12',
                         'KK_KUNDENTYP',
                         'D19_VERSAND_ANZ_24',
                         'D19_GESAMT_OFFLINE_DATUM',
                         'D19_KONSUMTYP',
                         'D19_GESAMT_ANZ_24',
                         'D19_VERSAND_ONLINE_DATUM',
                         'KBA05_BAUMAX',
                         'D19_GESAMT_ONLINE_DATUM',
                         'D19_VERSAND_DATUM']

    
    #df_parsed = df.copy()
    df.drop(columns_to_drop, axis = 1, inplace = True)
    
    
    print('====== Delete Columns ====')
    
    # Split dataframe 
    
    #df_copy = df_parsed.copy()
    df = df.dropna(thresh= 250) # Keep only the rows with at least 250 non-NA values
   
    
    print('======== Impute the missing values ======= ')
    
    
    df_most_freq_values_imputed = impute_values(df_parsed)
    
    
    print('====== Re-encode binary fature (OST_WEST_KZ) ======')
    
    bin_values = {'W': 1, 'O':0}
    df_most_freq_values_imputed['OST_WEST_KZ'] = df_most_freq_values_imputed['OST_WEST_KZ'].map(bin_values)
    
    print('======= Re-encode multi categorical features ========')
    
    to_reencode = ['CAMEO_DEU_2015',
                   'CAMEO_DEUG_2015',
                   'CAMEO_INTL_2015',
                   'D19_LETZTER_KAUF_BRANCHE']
    
    
    df_dummies = pd.get_dummies(df_most_freq_values_imputed, columns = to_reencode)
    
    print('======== Re-encode EINGEFUEGT_AM to year and month ======= ')
    
    df_dummies['EINGEFUEGT_AM'] = pd.to_datetime( df_dummies['EINGEFUEGT_AM'],
                                                 format = '%Y/%m/%d' )
                                                 
    df_dummies['EINGEFUEGT_AM'] = df_dummies['EINGEFUEGT_AM'].dt.year
    df_dummies['EINGEFUEGT_AM_month'] = df_dummies['EINGEFUEGT_AM'].dt.month
                                                 
    
    if columns is not None:
        
        diff = np.setdiff1d(columns, df_dummies.columns)
        #for column in diff:
        print(' Missing column:',diff)

            df_dummies[column] = 0.0
            df_dummies[column] = df_dummies[column].astype('float')

    
    
    return df_dummies

IndentationError: unexpected indent (<ipython-input-4-d85fc583f1a6>, line 118)

In [ ]:
def impute_values(df):
    
    df_copy = df.copy
    
    columns = df_copy.columns[df_copy.isnull().sum()]
    count = 0
    
    for column in columns:
        count += 1
        most_frequent_value = df_copy.groupby([column]).count().sort_values(ascending = False).index[0]
        
        df_copy[columns].fillna(most_frequent_value, inplace = True)
        
    return df_copy

**After analysing all of those features, I decided to do as follows:**
* **OST_WEST_KZ** binary feature that will bee re-encoded to 0 and 1
* **LNR** - It seems to be a index so we can get rid of it
* **CAMEO_DEU_2015** - This feature has many rows with 'XX' category (347 rows). I will consider those inputs as missing values and replace with a numerical value. The feature needs to be reencoded.
* **CAMEO_DEUG_2015** - Similarly the CAMEO_INTL_2015, this feature has many 'X' inputs. I will do the same as the above feature
* **CAMEO_INTL_2015** - Same as above feature
* **D19_LETZTER_KAUF_BRANCHE** - This featur has 36 categories (strings) and needs to be encoded
* **EINGEFUEGT_AM** - It is a date format feature and needs to be re-enconded (year e/or month)

In [ ]:

#display(pd.to_datetime(df_low_missing_values.EINGEFUEGT_AM, format="%Y-%m-%d").head())

In [ ]:
#df_low_missing_values['year'] = df_low_missing_values.EINGEFUEGT_AM.dt.year
#df_low_missing_values['month'] = df_low_missing_values.EINGEFUEGT_AM.dt.month


## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
#mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [ ]:
#mailout_train.head()

In [ ]:
#save_csv(mailout_train, 'data_project', 'Udacity_MAILOUT_052018_TRAIN.csv')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
#mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

In [ ]:
mailout_test.head()

In [ ]:
#save_csv(mailout_test, 'data_project', 'Udacity_MAILOUT_052018_TEST.csv')

In [ ]:
display(mailout_train.shape )
display(mailout_test.shape)

In [ ]:
mailout_train.CAMEO_INTL_2015.unique()